In [1]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
try:
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        print("notebook")
        from tqdm import tqdm_notebook as tqdm
    else:
        raise RuntimeError
except (NameError, RuntimeError):
    from tqdm import tqdm
import re
import MeCab
from gensim import corpora, models

notebook


In [2]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [3]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

In [4]:
def tokenizer(text):
    l = [line.split("\t") for line in mecab.parse(text).split("\n")]
    res = [i[0] for i in l 
                   if len(i) >=4 
                       and ("名詞" in i[3] or "動詞" in i[3] or "形容詞" in i[3] )
                       and "数" not in i[3] and "助動詞" not in i[3] 
                       #and not re.search("[0-9]", i[0])
                       and i[0] not in stopwords
            ]
    return res

In [5]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認め', '自分自身', '若さ故の過ち']

In [6]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/movie-enter'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/topic-news'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/peachy'),
 PosixPath('text/sports-watch'),
 PosixPath('text/dokujo-tsushin'),
 PosixPath('text/smax')]

In [7]:
articles = [a for categ in dirs for a in categ.iterdir()]

In [8]:
len(articles)

7376

In [9]:
def read_doc(doc_id):
    with articles[doc_id].open() as f:
        print(f.read())

In [10]:
class Doc_manager():
    def __init__(self, docs):
        self.docs = docs
        
    def read_doc(self, doc_id):
        with self.docs[doc_id].open() as f:
            print(f.read())

In [11]:
dm = Doc_manager(articles)

In [12]:
dm.read_doc(0)

http://news.livedoor.com/article/detail/5978741/
2011-10-30T10:15:00+0900
【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か
　2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初の長編サスペンス『八日目の蝉』。2010年に檀れいと北乃きいの出演によりテレビドラマ化された同作が、2011年4月に永作博美と井上真央の出演によって映画化。そして、劇場公開から半年が過ぎた10月28日、DVD＆ブルーレイとなって発売されました。

八日目の蝉
　妻子ある男と愛し合い、その子を身ごもりながら、あきらめざるをえなかった女。彼女は同時に、男の妻が子供を産んだことを知る。その赤ん坊を見に行った女は、突発的にその子を連れ去り、逃避行を続けた挙句、小豆島に落ち着き、母と娘として暮らしはじめる。


不倫相手の子供を誘拐し、4年間育てた女
　永作博美が演じる野々宮希和子は、不倫相手の子を宿しながらも、彼の「いずれ妻と別れるから、それまで待ってくれ」という常套句を信じて、中絶。後遺症により、二度と子供を産めない身体となってしまいます。その後、不倫相手から彼の妻が出産したことを知らされ、別れを決意。最後に諦めをつけるため、彼らの生後6ヶ月の赤ん坊・恵理菜の顔を見た希和子でしたが、自分に笑顔で向けた恵理菜を見て、思わず誘拐。名前を変えて恵理菜を薫と名付けると、人目を避けて各地を転々とし、二人で幸せな時間を過ごしますが、辿り着いた最後の場所・小豆島で4年の逃避行に終止符を打ちます。


誘拐犯に育てられた女
　4歳になって実の両親と再会を果たした後も、世間から言われの無い中傷を受け、本当の両親との関係を築けないまま、21歳の大学生へと成長した恵理菜。過去と向き合うことを避けてきた恵理菜でしたが、劇団ひとりが演じる不倫相手・岸田孝史の子を宿し、ずっと憎み続けてきた希和子と同じ道を歩んでいることに気付いた彼女は、小池栄子が演じるルポライター・安藤千草と共に、4年間の逃亡生活を追憶する旅に出ます。希和子との幸せだった時間に触れながら、最終地・小豆島に辿り着いた恵理菜が見た真実とは？


八日目の蝉は幸せなのだろうか？
　蝉は俗説として、一生の大半を幼虫として地下で費やし

In [13]:
docs = []
for a in tqdm(articles):
    with a.open() as f:
        f.readline()
        f.readline()
        docs.append(tokenizer(f.read()))

HBox(children=(IntProgress(value=0, max=7376), HTML(value='')))

In [14]:
d = corpora.Dictionary(docs)

In [15]:
d

In [16]:
d.filter_extremes(no_below=5, no_above=0.2)
# nob_below以下の個数の単語を無視
# no_aboveの割合以上に出てくる単語を無視

d.compactify()
# idを振り直してコンパクトにする。

In [17]:
dic_num = len(d)
dic_num

26238

In [18]:
d.doc2bow(["パソコン","テレビ","パソコン"])

[(1858, 1), (10447, 2)]

In [19]:
corpus = [d.doc2bow(w) for w in docs]

In [20]:
len(corpus)

7376

In [21]:
M = len(corpus)
V =  len(d)
k = 10

In [62]:
gamma_di = 1.0 
lambda_wi =  1.0

In [63]:
gamma_ = np.random.rand(M, k)
lambda_ = np.random.rand(V, k)
q_ = np.random.rand(M,V,k)

In [64]:
gamma_.shape

(7376, 10)

In [65]:
q_.shape

(7376, 26238, 10)

In [66]:
from scipy.special import digamma

In [73]:
for d in range(M):
    doc = corpus[d]
    N_d = len(doc)
    for n in range(N_d):
        gamma_sum = np.sum(gamma_, axis=1)
        lambda_sum = np.sum(lambda_, axis=0)
        w = int(doc[n][0])
        for i in range(k):
            gamma_di = gamma_[d,i]
            lambda_wi = lambda_[w,i]
            q_[d,w,i] = np.exp(digamma(gamma_di) - digamma(gamma_sum)[i] + digamma(lambda_wi) - digamma(lambda_sum)[i])
            gamma_[d,i] += np.sum(q_[:,:,i],axis=1)[d,i]
            lambda_[w,i] += np.sum(q_[:,:,i],axis=0)[w,i]
    break
alpha = 

[ 13129.29143328  13092.32879695  13088.91018018  13127.26894481
  13090.87641594  13142.27211325  13113.15931525  13159.3353723
  13159.20109692  13178.13428472]
0
0
0


KeyboardInterrupt: 

165

In [56]:
a = np.random.rand(2,5,4)

In [57]:
a

array([[[ 0.82161454,  0.84613451,  0.32539139,  0.49921134],
        [ 0.17964535,  0.16877479,  0.32385954,  0.16007882],
        [ 0.5554521 ,  0.90102776,  0.18467052,  0.37564718],
        [ 0.02271779,  0.64741619,  0.9545076 ,  0.81924748],
        [ 0.60562582,  0.15348833,  0.70630232,  0.09752852]],

       [[ 0.20903568,  0.14525822,  0.84354762,  0.30923472],
        [ 0.71600548,  0.79449694,  0.39432531,  0.96630687],
        [ 0.51587629,  0.22154471,  0.9504776 ,  0.42384227],
        [ 0.1957035 ,  0.63891291,  0.46439081,  0.13625856],
        [ 0.83550612,  0.83827079,  0.37492194,  0.8535355 ]]])

In [58]:
np.sum(a[],axis=1)

array([[ 2.1850556 ,  2.71684158,  2.49473137,  1.95171334],
       [ 2.47212706,  2.63848357,  3.02766328,  2.68917793]])